Look at the intensities

In [9]:
from src.data  import read_data

NameError: name 'plot_histogram' is not defined

In [7]:
# %% Run stuff
channel = "green"
padding = [[0, 0], [50, 500], [20, 20]]

pallium_path = Path("D:/Code/repos/psd95_segmentation/data/raw/pallium")
files = [x for x in pallium_path.glob('*tif') if x.is_file()]

num_prc = 11
prc_green = np.zeros((len(files), num_prc))

for count, file_name in enumerate(files):
    print(count)
    my_img = read_channel(channel, file_name.as_posix(), padding)
    prc_green[count, :] = get_percentiles(my_img)


# %% Run stuff
with open('prc_green.pickle', 'wb') as f:
    pickle.dump(prc_green, f)

# %% Run stuff
plot_histogram(prc_green[:, 6], 350, 50)

# %% Get Karl's csv
info_df = pd.read_csv('D:/Code/repos/psd95_segmentation/data/raw/karls_good2.csv')

# %% Get a slice with year info
info_year = info_df.loc[:, ['Source Image', 'Subject Issue Date']].sort_values(by=['Source Image'])
info_year.drop_duplicates(subset='Source Image', inplace=True)

# %%
path_text = files[0].as_posix().split(sep='/')[-1].split(sep='.')[0].split(sep='_')[-1]

# %% match files with year
years = np.zeros((len(files), 1))
names = []
for count, file_name in enumerate(files):
    img_name = get_image_name(file_name)
    year = info_year.loc[info_year['Source Image'] == img_name,
                         'Subject Issue Date'].values[0].split(sep='-')[0]
    print(f"#{count} image {img_name} year {year}")
    years[count] = year
    names.append(img_name)

# %% merge year w percentile
img_intensity_df = pd.DataFrame(np.concatenate((years, prc_green), axis=1),
                                columns=['Year', 'green_prc_0', 'green_prc_10',
                                         'green_prc_20', 'green_prc_30',
                                         'green_prc_40', 'green_prc_50',
                                         'green_prc_60', 'green_prc_70',
                                         'green_prc_80', 'green_prc_90',
                                         'green_prc_100'])

# %% merge year w percentile
from pandas.plotting import scatter_matrix

scatter_matrix(img_intensity_df, alpha=0.2, figsize=(6, 6), diagonal='kde')


# %%

# %%
fig = plot_boxplot(img_intensity_df, 'Year', 'green_prc_100')
print (type(fig))
# %%
fig.savefig('green_prc_100.png')

# %%
y_axis = 'nuc_prc_90'
fig = plot_boxplot(img_intensity_df, 'Year', y_axis)
fig.savefig(y_axis + '.png')

# %% Run stuff
channel = "red"
prc_red = np.zeros((len(files), num_prc))
for count, file_name in enumerate(files):
    print(count)
    my_img = read_channel(channel, file_name.as_posix(), padding)
    prc_red[count, :] = get_percentiles(my_img)

# %%
img_intensity_df = pd.DataFrame(np.concatenate((years, prc_green, prc_red), axis=1),
                                columns=['Year', 'green_prc_0', 'green_prc_10',
                                         'green_prc_20', 'green_prc_30',
                                         'green_prc_40', 'green_prc_50',
                                         'green_prc_60', 'green_prc_70',
                                         'green_prc_80', 'green_prc_90',
                                         'green_prc_100',
                                         'red_prc_0', 'red_prc_10',
                                         'red_prc_20', 'red_prc_30',
                                         'red_prc_40', 'red_prc_50',
                                         'red_prc_60', 'red_prc_70',
                                         'red_prc_80', 'red_prc_90',
                                         'red_prc_100'])
img_intensity_df.insert(0, 'Name', names, True)
# %%
feature = 'red_prc_0'
fig = plot_boxplot(img_intensity_df, 'Year', feature)
fig.savefig(feature + '.png')

# %%
info_df.sort_values(by="ID", inplace=True)
# %% match files with year
years = np.zeros((len(files), 1))

# %% match files with year

id_segments = []
timepoint = []
for count, file_name in enumerate(files):
    img_name = get_image_name(file_name)
    id_segment = info_df.loc[info_df['Source Image'] == img_name, 'ID'].values[0]
    print(f"#{count} image {img_name} ID {id_segment}")
    id_segments.append(id_segment)

    if id_segment[-2] == '6' or id_segment[-2] == '5':
        timepoint.append(2)
    elif id_segment[-2] == '3' or id_segment[-2] == '2':
        timepoint.append(1)
    else:
        raise ValueError('weird time point')

# %%
img_intensity_df.insert(1, 'tp', timepoint, True)

# %% download w redirect

link_head = 'https://synapse.isrd.isi.edu'
nuc_path = 'D:/Code/repos/psd95_segmentation/data/raw/csv/pallium/nuc/'
nuc_url = []
for count, file_name in enumerate(files):
    img_name = get_image_name(file_name)
    url = info_df.loc[info_df['Source Image'] == img_name, 'Segments Filtered URL'].iloc[0]
    if url.split('.')[1] == 'nuclei_only':
        url = link_head + url
        print(f"#{count} image {img_name} url {url}")

        nuc_url.append(url)

        nuc_csv = requests.get(url, allow_redirects=True)
        open(nuc_path + img_name + '_nuclei_only.csv', 'wb').write(nuc_csv.content)

# %% look at intensity of the nuclei
img_intensity_df = pd.read_csv('img_intensity_df.csv')
img_names = img_intensity_df.iloc[:, 1]

nuc_prc = np.zeros((len(img_names), 5))
for count, img_name in enumerate(img_names):
    print(f"#{count} image {img_name}")

    csv_file = nuc_path + img_name + '_nuclei_only.csv'
    nuc_csv_df = pd.read_csv(csv_file, skiprows=[1])

    nuc_intensity = nuc_csv_df.loc[:, ['raw core']]
    nuc_prc[count, :] = np.percentile(nuc_intensity, [10, 25, 50, 75, 90])

nuc_prc_df = pd.DataFrame(nuc_prc, columns=['nuc_prc_10', 'nuc_prc_25',
                                            'nuc_prc_50', 'nuc_prc_75', 'nuc_prc_90'])

img_intensity_df = pd.concat([img_intensity_df, nuc_prc_df], axis=1)

# %%
csv_file = nuc_path + img_name + '_nuclei_only.csv'
nuc_csv_df = pd.read_csv(csv_file, skiprows=[1])
# %% NOW DEALING WITH SYNAPSES
# load classified synapses

syn_path = 'D:/Code/repos/psd95_segmentation/data/raw/csv/pallium/syn/'
img_names = img_intensity_df.iloc[:, 1]
link_head = 'https://synapse.isrd.isi.edu'

for count, img_name in enumerate(img_names):
    segmentations = info_df.loc[info_df['Source Image'] == img_name,
                                ['RID', 'classifier_name', 'Segments Filtered URL']]
    for index, segmentation in segmentations.iterrows():
        rid = segmentation[0]
        cl_name = segmentation[1]
        url = segmentation[2]
        csv_type = url.split('.')[1]
        if csv_type == 'synapses_only' or csv_type == 'segments_only':
            url = link_head + url
            syn_csv = requests.get(url, allow_redirects=True)
            open(syn_path + img_name + '_' + rid + '_' + csv_type + '.csv',
                 'wb').write(syn_csv.content)
# %% NOW DEALING WITH SYNAPSES
# load classified synapses

syn_path = 'D:/Code/repos/psd95_segmentation/data/raw/csv/pallium/syn/'
img_names = img_intensity_df.iloc[:, 1]
syn_prc = np.zeros((len(img_names), 5))
img_w_two = []

for count, img_name in enumerate(img_names):
    num_syn = 0
    segmentations = info_df.loc[info_df['Source Image'] == img_name,
                                ['RID', 'classifier_name', 'Segments Filtered URL']]
    for index, segmentation in segmentations.iterrows():
        rid = segmentation[0]
        cl_name = segmentation[1]
        url = segmentation[2]
        csv_type = url.split('.')[1]
        if csv_type == 'synapses_only' or csv_type == 'segments_only':
            csv_file = syn_path + img_name + '_' + rid + '_' + csv_type + '.csv'
            if Path(csv_file).exists():
                num_syn = num_syn + 1
                if num_syn > 1:
                    img_w_two.append(img_name)
                    print(f"{img_name} {num_syn}")

# %% Add percentiles for synapses
syn_prc = np.zeros((len(img_names), 5))

for count, img_name in enumerate(img_names):
    done = 0
    segmentations = info_df.loc[info_df['Source Image'] == img_name,
                                ['RID', 'classifier_name', 'Segments Filtered URL']]
    for index, segmentation in segmentations.iterrows():
        rid = segmentation[0]
        cl_name = segmentation[1]
        url = segmentation[2]
        csv_type = url.split('.')[1]
        if csv_type == 'synapses_only':
            csv_file = syn_path + img_name + '_' + rid + '_' + csv_type + '.csv'
            if Path(csv_file).exists() and done == 0:
                done = 1
                syn_csv_df = pd.read_csv(csv_file, skiprows=[1])
                syn_intensity = syn_csv_df.loc[:, ['raw core']]
                syn_prc[count, :] = np.percentile(syn_intensity,
                                                  [10, 25, 50, 75, 90])

syn_prc_df = pd.DataFrame(syn_prc, columns=['syn_prc_10', 'syn_prc_25',
                                            'syn_prc_50', 'syn_prc_75', 'syn_prc_90'])

img_intensity_df = pd.concat([img_intensity_df, syn_prc_df], axis=1)

# %%
feature = 'syn_prc_90'
fig = plot_boxplot(img_intensity_df, 'Year', feature)
fig.savefig(feature + '.png')

# %%

# %% save table
img_intensity_df.to_csv('intensity_over_years.csv')



NameError: name 'read_channel' is not defined